# Classification Predict

### Climate change tweet classsification 

This notebook is aimed at classifying if a person/custumer believe in climate change or not, this is to help companies with determining if customising their product for lessining carbon footprint and environmental impact, is a good idea or not.

## Importing the modules

Start by importing packages that will be needed

In [1]:
import nltk

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# set plot style
sns.set()

import warnings
warnings.filterwarnings('ignore')

import string
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nlppreprocess import NLP
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier


Checking the fields of our data

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_with_no_labels.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## Text to numeric

Since the model only recognise numeric data for predictions, the text data has to be converted to numeric data type and this is achieved using text frequency - inverse document frequency (TF-IDF).

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [4]:
#TF-IDF for coverting text to numeric data type
data = train_df['message']

vectorizer=TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(data)

In [5]:
test_data = test_df['message']

vectorized_t = vectorizer.transform(test_data)
X_test_t = vectorized_t

In [6]:
# Arranging the data into predictor variables and label
X = X_vectorized
y = train_df['sentiment']

## Models

#### NB: Splitting data in this case is just for checking the classification report  but the submissions on kaggle are predicted without splitting the data

Spliting the training data into test and training, or test and validation sets

In [7]:
from sklearn.model_selection import train_test_split

# Splitting the training data into 95% training and 5% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

### Logistic regression

#### hyperparameter tunning

In [ ]:
model = LogisticRegression()
solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,C=c_values)
grid_search = GridSearchCV(estimator=model, param_grid=grid, scoring='f1')
grid_result = grid_search.fit(X_train,y_train)

In [ ]:
grid_result.best_params_

#### Checking the F1 score and classification report of the split data

In [16]:
lr = LogisticRegression(C= 100, solver = 'liblinear', random_state =42)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

print('F1_score: ',round(metrics.f1_score(y_test,y_pred, average = 'weighted'),3))
print("\n\nClassification Report:\n\n", metrics.classification_report(y_test, y_pred))

F1_score:  0.76


Classification Report:

               precision    recall  f1-score   support

          -1       0.70      0.51      0.59        61
           0       0.61      0.47      0.53       123
           1       0.80      0.86      0.83       441
           2       0.79      0.84      0.82       166

    accuracy                           0.77       791
   macro avg       0.73      0.67      0.69       791
weighted avg       0.76      0.77      0.76       791



#### making prediction for unsplit data

In [17]:
lr.fit(X,y)
y_pred_t = lr.predict(X_test_t)

## Saving the predictions

Tunning the predictions into a dataframe

In [18]:
submission = pd.DataFrame({'tweetid':test_df['tweetid'],
                          'sentiment':y_pred_t})

copying the predictions into a CSV file

In [ ]:
submission.to_csv('classification6.csv',index=False)

Use the above saved CSV file on Kaggle for submission

## Saving the model

In [ ]:
import pickle
model_save_path = "lr25.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(lr,file)